## Requirements

This notebook needs some extra setup as it uses packages not included with the standard install, `lsst_distrib`.

Make sure to set up the LSST software first.  This is necessary to make sure the next modules install relative to the kernel python version:

```
source /opt/lsst/software/stack/loadLSST.bash
setup lsst_distrib
```

Next -- pip install dependencies:

```
pip install --user pyarrow
pip install --user paramnb
```

Then install the LSST software:

* `pipe_analysis`
    ```
    cd ~/repos
    git clone https://github.com/lsst-dm/pipe_analysis
    cd pipe_analysis
    setup -k -r .
    scons
    ```


* `qa_explorer`
    ```
    cd ~/repos
    git clone https://github.com/lsst-dm/qa_explorer
    cd qa_explorer
    git checkout tickets/DM-14521
    setup -k -r .
    scons
    ```
    
Add entries to `~/notebooks/.user_setups`:

```
echo 'setup -r ~/repos/pipe_analysis' >> ~/notebooks/.user_setups
echo 'setup -r ~/repos/pipe_analysis' >> ~/notebooks/.user_setups
```

import holoviews as hv
hv.notebook_extension('bokeh')

In [ ]:
from lsst.daf.persistence import Butler
butler = Butler('/datasets/hsc/repo/rerun/RC/w_2018_17/DM-14055/')

In [ ]:
from lsst.qa.explorer.catalog import CoaddCatalog, VisitCatalog, MultiMatchedCatalog
from lsst.qa.explorer.utils import get_visits

tract = 9615
filt = "HSC-R"
dataId = {'tract':tract, 'filter':filt}
catalog = CoaddCatalog(butler, dataId)

In [ ]:
# Modify this cell to calculate whatever functors you want

from lsst.qa.explorer.functors import MagDiff,Seeing,Column
cmodel = MagDiff('modelfit_CModel', 'base_PsfFlux')
gauss = MagDiff('base_GaussianFlux', 'base_PsfFlux')
count = Column('base_InputCount_value')
seeing = Seeing()

funcs = {'cmodel':cmodel,'gauss':gauss,'count':count,'seeing':seeing,}

In [ ]:
from lsst.qa.explorer.dataset import QADataset
flags = ['calib_psfUsed',
'qaBad_flag',
'merge_measurement_i',
'merge_measurement_r',
'merge_measurement_z',
'merge_measurement_y',
'merge_measurement_g',
'base_Footprint_nPix_flag',
'base_PixelFlags_flag_inexact_psfCenter']

data = QADataset(catalog, funcs, flags=flags)

In [ ]:
# Calculate dataframe; see how long it takes.
%time data.df.head()

In [ ]:
from lsst.qa.explorer.plots import FilterStream, multi_scattersky
filter_stream = FilterStream()
multi_scattersky(data.ds.groupby('label'), filter_stream=filter_stream, width=900, height=300)

In [ ]:
from lsst.qa.explorer.plots import FlagSetter
import paramnb

flag_setter = FlagSetter(filter_stream=filter_stream, flags=data.flags, bad_flags=data.flags)
paramnb.Widgets(flag_setter, callback=flag_setter.event, push=False, on_init=True)

In [ ]:
%%output max_frames=10000
%%opts Points [width=500, height=500, tools=['hover'], colorbar=True] (cmap='coolwarm', size=4)

# Change dimension to whichever you would like to explore
dimension = 'cmodel'
coadd_dmap = data.coadd_explore(dimension, filter_stream=filter_stream).relabel(dimension)
tap = hv.streams.Tap(source=coadd_dmap, rename={'x':'ra', 'y':'dec'})

coadd_dmap

In [ ]:
import lsst.afw.display as ad

In [ ]:
ad.setDefaultBackend("ginga")

from lsst.qa.explorer.display import CoaddDisplay
coadd_display = CoaddDisplay(butler=butler, filt=filt, dims=(500,500))
coadd_display.connect_tap(tap)
coadd_display.embed()

In [ ]:
%%opts RGB [width=400, height=400]

from lsst.qa.explorer.plots import skyplot, skyplot_layout
skyplots = [skyplot(data.ds.groupby('label'), vdim=d, filter_stream=filter_stream).relabel(d) for d in funcs]
skyplot_layout(skyplots).cols(3)
    